# Basic Model Interface (BMI) for streamflow routing using t-route

This t-route network was developed for use in the Next Generation National Water Model (Nextgen). Nextgen runs models with Basic Model Interface (BMI).

In [32]:
import sys

sys.path.append("src/")
import bmi_troute # This is the BMI t-route that we will be running from the file: bmi_troute.py 
import nwm_routing.__main__ as tr
#import troute.nhd_preprocess as nhd_prep # This is used to read q_lateral data from files, won't be needed when t-route bmi is run with model engine

In [33]:
model = bmi_troute.bmi_troute()
model.initialize(bmi_cfg_file='test/BMI/test_MC_bmi.yaml')

In [34]:
model.get_time_step()

300

In [35]:
run_sets = model._network.build_forcing_sets()
model._network.assemble_forcings(run_sets[0],)



In [36]:
model.set_value('land_surface_water_source__volume_flow_rate', model._network.qlateral.to_numpy())
#model.set_value('coastal_boundary__depth', coastal_boundary_depth_df.to_numpy())
#model.set_value('usgs_gage_observation__volume_flow_rate', data_assimilation.reservoir_usgs_df.to_numpy())
#model.set_value('usace_gage_observation__volume_flow_rate', src)
#model.set_value('rfc_gage_observation__volume_flow_rate', src)

In [37]:
model.update()

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1098: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("lastobs_discharge", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1099: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("time_since_lastobs", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),


In [38]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

10236    1494929.000
10237    1460805.000
10245    1079974.375
Name: (11, q), dtype: float32

In [39]:
model.get_current_time()

3600.0

In [42]:
model.initialize(bmi_cfg_file='test/BMI/test_MC_bmi.yaml')

In [43]:
run_sets = model._network.build_forcing_sets()
model._network.assemble_forcings(run_sets[0],)

In [44]:

model.set_value('land_surface_water_source__volume_flow_rate', model._network.qlateral.to_numpy())
#model.set_value('coastal_boundary__depth', coastal_boundary_depth_df.to_numpy())
#model.set_value('usgs_gage_observation__volume_flow_rate', src)
#model.set_value('usace_gage_observation__volume_flow_rate', src)
#model.set_value('rfc_gage_observation__volume_flow_rate', src)

In [45]:
model.update_until(1800) # input value is seconds. 1800s = 0.5 hours

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1098: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("lastobs_discharge", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1099: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("time_since_lastobs", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),


In [46]:
model.get_current_time()

1800.0

In [47]:

model.get_value('channel_exit_water_x-section__volume_flow_rate')

10236    150371.281250
10237    137978.265625
10245    309930.093750
Name: (5, q), dtype: float32

In [48]:

model.update()

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1098: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("lastobs_discharge", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:1099: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lastobs_df_sub.get("time_since_lastobs", pd.Series(index=lastobs_df_sub.index, name="Null")).values.astype("float32"),


In [49]:
model.get_current_time()

3600.0

In [50]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

10236    1494929.000
10237    1460805.000
10245    1079974.375
Name: (5, q), dtype: float32

In [51]:
model.finalize()